In [ ]:
!pip install transformers datasets scikit-learn torch # Installing dependencies

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
!pip uninstall -y datasets


Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4


In [ ]:
!pip install datasets==2.16.0 # This version is known to work with common NumPy versions

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
!pip install -U transformers


In [ ]:
!pip install -U datasets


In [ ]:
import transformers
print(transformers.__version__)


4.52.3


In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch

In [ ]:
fake = pd.read_csv("/content/drive/My Drive/Fake.csv") #fake news

In [ ]:
true = pd.read_csv("/content/drive/My Drive/True.csv") #true news

In [ ]:
#Setting labels
fake['label'] = 1
true['label'] = 0

In [ ]:
#Combining data
data = pd.concat([fake, true], ignore_index=True)
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
data['text'] = data['title'] + ' ' + data['text']
data = data[['text', 'label']]

In [ ]:
data = data.sample(n=3000, random_state=42)  # try 3k samples max

In [ ]:
print(data.columns)

Index(['text', 'label'], dtype='object')


In [ ]:
#Split and Convert to Hugging Face Dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['text'].tolist(), data['label'].tolist(), test_size=0.2, random_state=42)


In [ ]:
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})


In [ ]:
#Tokenization
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
# Format Dataset for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [ ]:
training_args = TrainingArguments(
    output_dir="./bert-fake-news",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

In [ ]:
# Evaluation Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


<ipython-input-20-48c5c4e00723>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
!pip install -U datasets numpy #Erros may come up due to package's versions

In [ ]:
import os
os.environ["DATASETS_ENABLE_INSTANT_REPLACE"] = "false"
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.053200,0.009408,0.998333
2,0.001200,0.009695,0.998333
3,0.000900,0.010496,0.998333


TrainOutput(global_step=450, training_loss=0.03811273777857423, metrics={'train_runtime': 382.8969, 'train_samples_per_second': 18.804, 'train_steps_per_second': 1.175, 'total_flos': 953765270323200.0, 'train_loss': 0.03811273777857423, 'epoch': 3.0})

In [ ]:
# Evaluate
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
print("Classification Report:\n", classification_report(test_labels, preds))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       290
           1       1.00      1.00      1.00       310

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



In [ ]:
results = trainer.evaluate()
print("Accuracy:", results["eval_accuracy"])


Accuracy: 0.9983333333333333


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model.save_pretrained("/content/fake-news-bert")
tokenizer.save_pretrained("/content/fake-news-bert")


('/content/fake-news-bert/tokenizer_config.json',
 '/content/fake-news-bert/special_tokens_map.json',
 '/content/fake-news-bert/vocab.txt',
 '/content/fake-news-bert/added_tokens.json',
 '/content/fake-news-bert/tokenizer.json')

In [ ]:
# Load model & tokenizer from the local path
local_model_path = "/content/fake-news-bert"
model = AutoModelForSequenceClassification.from_pretrained(local_model_path)
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Now you can use 'model' and 'tokenizer' for predictions

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Load model & tokenizer
# The model was saved in the previous step, so it's loaded from the local path
model = AutoModelForSequenceClassification.from_pretrained("/content/fake-news-bert")
tokenizer = AutoTokenizer.from_pretrained("/content/fake-news-bert")

def predict_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Remove 'token_type_ids' if it exists, as DistilBERT models don't use it
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).detach().numpy()[0]
    label = np.argmax(probs)
    confidence = float(probs[label]) * 100

    return "Real" if label == 0 else "Fake", round(confidence, 2)


In [ ]:
text = "India launches next-gen weather satellite"
label, conf = predict_news(text)
print(f"Prediction: {label} ({conf}%)")


Prediction: Real (93.71%)


In [ ]:
!ls /content/fake-news-bert

config.json	   special_tokens_map.json  tokenizer.json
model.safetensors  tokenizer_config.json    vocab.txt
